##### **Note:** *Suggestions and Advice are very much welcome and will be sincerely appreciated* 😃

## Import relevant libraries

In [1]:
import pandas as pd
import numpy as np
import glob
import warnings
warnings.simplefilter("ignore", pd.core.common.SettingWithCopyWarning)

In [2]:
# load the filepaths to all the .csv files in a list
filepath_list = glob.glob(
    "/kaggle/input/unlock-profits-with-e-commerce-sales-data/*.csv")

# extract and store file names without spaces in a list
# this is to use for assigning dataframes
file_list = [element.split("data/")[1]for element in filepath_list]
file_list = ["".join(element.split(" ")).split(".")[0].replace("-","").lower()
                       for element in file_list] 


# extract the datasets from parent location and assigned 
# them to the corresponding dataframe variables from file_list
for element in filepath_list:  
    # read to pandas dataframe from  element in filepath_list to df
    df = pd.read_csv(element, low_memory=False)

    # assign the dataframe df to variable with corresponding name
    globals()[
        file_list[filepath_list.index(element)]
    ] = df 

print(f"The names of all {len(file_list)} pandas dataframes are:\n")

for dataframe in file_list:
      print(dataframe)

The names of all 7 pandas dataframes are:

amazonsalereport
cloudwarehousecompersionchart
may2022
plmarch2021
internationalsalereport
expenseiigf
salereport


### <br>Function for cleaning the dataset/dataframe "amazonsalereport"

In [3]:
def wrangle_amazonsalereport(dataframe):
    
    # copy dataframe to df for wrangling
    df = dataframe.copy()
    
    #clean the column names
    col = [element.lower().replace(" ","").replace("-","") for element in df.columns]
    df.columns = col

    # change the dtype of col "Date" to datetime dtype
    df["date"] = pd.to_datetime(df["date"])
    
    # drop the redundant cols in the dataset
    df.drop(["index","fulfilledby","currency","unnamed:22","promotionids","courierstatus","shipcountry"],
            axis ="columns", inplace = True)
    
    # rename col "amount" to "amount(Rs.)"
    df.rename(columns = {"amount":"amount(Rs.)"}, inplace = True)
    
    # fill value "unknown" in rows where location (city and state) is not known 
    df["shipstate"].fillna("unknown", inplace = True)
    df["shipcity"].fillna("unknown", inplace = True)
    
    # fill value 0 in rows where postalcode is null
    df["shippostalcode"].fillna(0, inplace = True)
    
    #change dtype of postalcode to object
    df["shippostalcode"] = df["shippostalcode"].astype(int).astype(object)
    
    # clean the col shipstate to atain a 37 unique values i.e.
    # (28 states + 8 UT + 1 as "UNKNOWN")
    df["shipstate"] = df["shipstate"].str.upper()
    df["shipstate"].replace({"PONDICHERRY":"PUDUCHERRY","RAJSHTHAN":"RAJASTHAN","RAJSTHAN":"RAJASTHAN",
                              "RJ":"RAJASTHAN","PB":"PUNJAB","PUNJAB/MOHALI/ZIRAKPUR":"PUNJAB",
                              "ORISSA":"ODISHA","DELHI":"NEW DELHI","NL":"UNKNOWN","APO":"UNKNOWN",
                              "AR":"UNKNOWN"}, inplace = True)
    
    # change the char in col "category" to uppercase 
    df["category"] = df["category"].str.upper()
    
    # qty and amount value "0" is illogical in any order purchased
    # therefore, replace 0s in columns "qty" and "amount" with "NaN"
    df[["amount(Rs.)","qty"]] = df[["amount(Rs.)","qty"]].replace(0,np.nan)
    
    # drop duplicate rows,if any
    df.drop_duplicates(inplace = True)
    
    # return cleaned data
    return df.reset_index(drop = True)

### <br>Check the function ***wrangle_amazonsalereport()*** by printing head()

In [4]:
amazonsalereport_clean = wrangle_amazonsalereport(amazonsalereport)
amazonsalereport_clean.info()
amazonsalereport_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128969 entries, 0 to 128968
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   orderid           128969 non-null  object        
 1   date              128969 non-null  datetime64[ns]
 2   status            128969 non-null  object        
 3   fulfilment        128969 non-null  object        
 4   saleschannel      128969 non-null  object        
 5   shipservicelevel  128969 non-null  object        
 6   style             128969 non-null  object        
 7   sku               128969 non-null  object        
 8   category          128969 non-null  object        
 9   size              128969 non-null  object        
 10  asin              128969 non-null  object        
 11  qty               116165 non-null  float64       
 12  amount(Rs.)       118834 non-null  float64       
 13  shipcity          128969 non-null  object        
 14  ship

,orderid,date,status,fulfilment,saleschannel,shipservicelevel,style,sku,category,size,asin,qty,amount(Rs.),shipcity,shipstate,shippostalcode,b2b
0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,SET,S,B09KXVBD7Z,NaN,647.62,MUMBAI,MAHARASHTRA,400081,False
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,KURTA,3XL,B09K3WFS32,1.0,406.00,BENGALURU,KARNATAKA,560085,False
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,KURTA,XL,B07WV4JV4D,1.0,329.00,NAVI MUMBAI,MAHARASHTRA,410210,True
3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,WESTERN DRESS,L,B099NRCT7B,NaN,753.33,PUDUCHERRY,PUDUCHERRY,605008,False
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,TOP,3XL,B098714BZP,1.0,574.00,CHENNAI,TAMIL NADU,600073,False


### Export the clean data to csv file named **"amazon_sale_report_clean.csv"** in the current directory

In [5]:
amazonsalereport_clean.to_csv("amazon_sale_report_clean.csv", index = False)

### <br>Function for cleaning the dataset/dataframe **internationalsalereport**

In [6]:
def wrangle_internationalsalereport(dataframe):
    
    # copy dataframe into df
    df = dataframe.copy()
    
    # drop rows with nan
    df = df.dropna()
    
    # drop col name "index"
    df.drop("index", axis = "columns", inplace = True)
    
    # two dataframes,one with rows with mismatched cols and other without mismatched cols
    mismatched_df = df[(df["DATE"].str.slice(-1) == "1") |
                        (df["DATE"].str.slice(-1) == "2") == False].reset_index(drop = True)
    proper_df = df[(df["DATE"].str.slice(-1) == "1") |
                    (df["DATE"].str.slice(-1) == "2") == True].reset_index(drop = True)
    
    # correct the column names of the mismatched_df and drop the first row (redundant)
    mismatched_df.columns = ['CUSTOMER', 'DATE', 'Months', 'Style', 'SKU', 'PCS', 'RATE', 'GROSS AMT',
       'Stock']
    mismatched_df = mismatched_df.drop(0)
    
    #clean the col "Stock" 
    mismatched_df["Stock"] = (mismatched_df["Stock"].astype(str)
                              .str.split(".",expand = True).iloc[:,0].astype(int))
    
    # append distorted_df to proper_df: in this case it's pandas concat()
    df = pd.concat([proper_df,mismatched_df])
    
    # clean the cols names
    df.columns = [element.lower().replace(" ","")for element in df.columns]
    
    # change the dtype of date to datetime
    df["date"] = pd.to_datetime(df["date"])
    
    #change the datatype of pcs, rate and grossamt
    df[["rate","grossamt"]] = df[["rate","grossamt"]].astype(float)
    df["pcs"] = df["pcs"].astype(str).str.split(".",expand = True).iloc[:,0].astype(int)
    
    # drop duplicates,if any
    df.drop_duplicates(inplace = True)
    
    #change the dtype of Dat
    return df.reset_index(drop = True)
    

### <br>Check the function ***wrangle_internationalsalereport()*** by printing head()

In [7]:
internationalsalereport_clean = wrangle_internationalsalereport(internationalsalereport)
internationalsalereport_clean.info()
internationalsalereport_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22115 entries, 0 to 22114
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      22115 non-null  datetime64[ns]
 1   months    22115 non-null  object        
 2   customer  22115 non-null  object        
 3   style     22115 non-null  object        
 4   sku       22115 non-null  object        
 5   size      10943 non-null  object        
 6   pcs       22115 non-null  int64         
 7   rate      22115 non-null  float64       
 8   grossamt  22115 non-null  float64       
 9   stock     11172 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 1.7+ MB


,date,months,customer,style,sku,size,pcs,rate,grossamt,stock
0,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-L,L,1,616.56,617.0,NaN
1,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XL,XL,1,616.56,617.0,NaN
2,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XXL,XXL,1,616.56,617.0,NaN
3,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5009,MEN5009-KR-L,L,1,616.56,617.0,NaN
4,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5011,MEN5011-KR-L,L,1,616.56,617.0,NaN


### Export the clean data to csv file named ***"international_sale_report_clean.csv"*** in the current directory

In [8]:
internationalsalereport_clean.to_csv("international_sale_report_clean.csv", index = False)

### <br>Function for cleaning the dataset/dataframe **salereport**

In [9]:
def wrangle_salereport(dataframe):
    
    #copy data. Also drop nan
    df = dataframe.copy()
    df.dropna(inplace = True)
    
    #drop redundant column "index"
    df.drop("index", axis = "columns",inplace = True)
    
    # rename the col names
    df.columns = [element.replace(" ","").lower() for element in df.columns]
    df = df.rename(columns = {"skucode":"sku"})
    
    #change the dtype of the column "Stock"
    df["stock"] = df["stock"].astype(str).str.split(".",expand = True).iloc[:,0].astype(int)
    
    # drop duplicates,if any
    df.drop_duplicates(inplace = True)
    
    #reset the index and return the dataframe    
    return df.reset_index(drop = True)

### Check the function ***wrangle_salereport()*** by printing head()

In [10]:
salereport_clean = wrangle_salereport(salereport)
salereport_clean.info()
salereport_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9185 entries, 0 to 9184
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sku        9185 non-null   object
 1   designno.  9185 non-null   object
 2   stock      9185 non-null   int64 
 3   category   9185 non-null   object
 4   size       9185 non-null   object
 5   color      9185 non-null   object
dtypes: int64(1), object(5)
memory usage: 430.7+ KB


,sku,designno.,stock,category,size,color
0,AN201-RED-L,AN201,5,AN : LEGGINGS,L,Red
1,AN201-RED-M,AN201,5,AN : LEGGINGS,M,Red
2,AN201-RED-S,AN201,3,AN : LEGGINGS,S,Red
3,AN201-RED-XL,AN201,6,AN : LEGGINGS,XL,Red
4,AN201-RED-XXL,AN201,3,AN : LEGGINGS,XXL,Red


### Export the cleaned data to csv file named **"sale_report_clean.csv"** in the current directory

In [11]:
salereport_clean.to_csv("sale_report_clean.csv", index = False)

### <br>Function for cleaning the dataset/dataframe **cloudwarehousecompersionchart**

In [12]:
def wrangle_cloudwarehousecompersionchart(dataframe):
    
    # copy dataframe to pandas df
    df = dataframe.copy()
    
    # select only the relevant rows
    df = df.iloc[:5,:]
    
    # drop redundant col named "index"
    df.drop("index", axis = 1,inplace = True)
    
    # rename the col names with relevant names
    col = df.columns[[0,2]].tolist()      # list containing 'Shiprocet' and 'INCREFF'
    col = [str(element) + "(Rs./Unit)" for element in col]
    col.insert(0,df.loc[0][0])         # insert 'Heads' as first element in the list col
    df.columns= col                    # remane the col names
    
    # drop redundant first row
    df.drop(0,inplace = True)
    
    # clean the col Shiprocket and change dtype to float
    df["Shiprocket(Rs./Unit)"] = df["Shiprocket(Rs./Unit)"].str.slice(start = 1).astype(float)
    
    # clean the dirty cell in "INCREFF" column to extract numeric chars "0.15"
    first_char = df["INCREFF(Rs./Unit)"].str.slice(stop = 1)  # series of extract of first character
    string = [str(element) for element in range(9)] # list containing range(0,9) as string dtype
    condition = (first_char.isin(string))  # check first char is in list "string"
    condition = condition == False         # check first char is not in list "string"
    # slice the cell to get desired chars
    df["INCREFF(Rs./Unit)"][condition] = df["INCREFF(Rs./Unit)"][condition].str.slice(2,7) 
    
    
    # clean the col INCREFF and change dtype to float
    df["INCREFF(Rs./Unit)"] = df["INCREFF(Rs./Unit)"].astype(float)
    
    # drop duplicates,if any
    df.drop_duplicates(inplace = True)
    
    #return
    return df.reset_index(drop = True)

### <br>Check the function ***wrangle_cloudwarehousecompersionchart()*** by printing head()

In [13]:
# ignore unwanted warnings on the output
warnings.simplefilter("ignore", pd.core.common.SettingWithCopyWarning)

cloudwarehousecompersionchart_clean = wrangle_cloudwarehousecompersionchart(cloudwarehousecompersionchart)
cloudwarehousecompersionchart_clean.info()
cloudwarehousecompersionchart_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Heads                 4 non-null      object 
 1   Shiprocket(Rs./Unit)  4 non-null      float64
 2   INCREFF(Rs./Unit)     4 non-null      float64
dtypes: float64(2), object(1)
memory usage: 224.0+ bytes


,Heads,Shiprocket(Rs./Unit),INCREFF(Rs./Unit)
0,Inbound (Fresh Stock and RTO),4.0,4.00
1,Outbound,7.0,11.00
2,Storage Fee/Cft,25.0,0.15
3,Customer Return with Detailed QC,6.0,15.50


### <br>Export the cleaned data to csv file named **"cloudwarehousecompersionchart_clean.csv"** in the current directory

In [14]:
cloudwarehousecompersionchart_clean.to_csv("cloudwarehousecompersionchart_clean.csv",index = False)

### <br>Function for cleaning the dataset/dataframe **espenseiigf**

In [15]:
def wrangle_expenseiigf(dataframe): 
    
    # copy data to dataframe df
    df = dataframe.copy()
    
    # drop redundant col "index"
    df.drop("index", axis = "columns", inplace = True)
    
    # rename col names
    df.columns = ["receivedparticular","receivedamount(Rs.)","expenseparticular","expenseamount(Rs.)"]
    
    # drop first row (redundant data)
    df.drop(0,inplace = True)
    
    # fill null with proper values
    df["expenseparticular"].fillna("Total", inplace = True)
    df["receivedamount(Rs.)"].fillna(0, inplace = True)
    df["receivedparticular"].fillna("unknown", inplace = True)
    
    # change the dtype of both amount cols into float
    df[["receivedamount(Rs.)","expenseamount(Rs.)"]] = df[["receivedamount(Rs.)","expenseamount(Rs.)"]].astype(float)
    
    #return with index reset
    return df

### <br>Check the function ***wrangle_expenseiigf()*** by printing head()

In [16]:
expenseiigf_clean = wrangle_expenseiigf(expenseiigf)
expenseiigf_clean.info()
expenseiigf_clean.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 1 to 16
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   receivedparticular   16 non-null     object 
 1   receivedamount(Rs.)  16 non-null     float64
 2   expenseparticular    16 non-null     object 
 3   expenseamount(Rs.)   16 non-null     float64
dtypes: float64(2), object(2)
memory usage: 640.0+ bytes


,receivedparticular,receivedamount(Rs.),expenseparticular,expenseamount(Rs.)
1,06-19-22,1000.0,Large Bag,380.0
2,06-20-22,1500.0,"Stationary(Soft Pin, Paper pin for Dupatta, Fe...",170.0
3,06-22-22,500.0,OLA,839.0
4,06-23-22,2000.0,Auto Rent,520.0
5,unknown,0.0,TT Fine,500.0


### <br>Export the cleaned data to csv file named **"expenseiigf_clean.csv"** in the current directory

In [17]:
expenseiigf_clean.to_csv("expenseiigf_clean.csv", index = False)

### <br>Function for cleaning the dataset/dataframe **may2022**

In [18]:
def wrangle_may2022(dataframe,null = True):
    
    # copy dataframe to df
    df = dataframe.copy()
    
    # drop redundant col named "index"
    df.drop("index",axis = 1, inplace = True)
    
    # rename the column names
    df.columns = [element.replace(" ","_").lower() for element in df.columns]
    
    # dealing with "Nill" and "#VALUE!" entries in possible float-dtype columns
    
    # first: replace "Nill" and "#VALUE!" with integer 0 and change dtype to float
    df[['weight', 'tp', 'mrp_old','final_mrp_old',
    'ajio_mrp', 'amazon_mrp', 'amazon_fba_mrp',
    'flipkart_mrp', 'limeroad_mrp', 'myntra_mrp',
    'paytm_mrp','snapdeal_mrp']] = df[['weight', 'tp', 'mrp_old','final_mrp_old',
    'ajio_mrp', 'amazon_mrp', 'amazon_fba_mrp',
    'flipkart_mrp', 'limeroad_mrp', 'myntra_mrp',
    'paytm_mrp','snapdeal_mrp']].replace(["Nill","#VALUE!"],0).astype(float)
    
    # replace both 0 and "Nill" to "NaN" in all cols
    df = df.replace([0,"Nill"],np.nan)
    
    # change the values in "category" col to uppercase
    df["category"] = df["category"].str.upper()
    df["sku"] = df["sku"].str.upper()
    df["style_id"] = df["style_id"].str.upper()
    
    # drop duplicates, if any
    df.drop_duplicates(inplace = True)
    
    # two return options: a dataframe with null and a dataframe without null
    if null:
        return df.reset_index(drop = True)
    else:
        return df.dropna().reset_index(drop = True)

### <br>Check the function ***wrangle_may2022() with null*** by printing head()

In [19]:
may2022_clean_with_null = wrangle_may2022(may2022,null = True)
may2022_clean_with_null.info()
may2022_clean_with_null.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330 entries, 0 to 1329
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sku             1330 non-null   object 
 1   style_id        1330 non-null   object 
 2   catalog         1257 non-null   object 
 3   category        1257 non-null   object 
 4   weight          1257 non-null   float64
 5   tp              1324 non-null   float64
 6   mrp_old         1293 non-null   float64
 7   final_mrp_old   1293 non-null   float64
 8   ajio_mrp        1293 non-null   float64
 9   amazon_mrp      1293 non-null   float64
 10  amazon_fba_mrp  1293 non-null   float64
 11  flipkart_mrp    1293 non-null   float64
 12  limeroad_mrp    1293 non-null   float64
 13  myntra_mrp      1299 non-null   float64
 14  paytm_mrp       1293 non-null   float64
 15  snapdeal_mrp    1293 non-null   float64
dtypes: float64(12), object(4)
memory usage: 166.4+ KB


,sku,style_id,catalog,category,weight,tp,mrp_old,final_mrp_old,ajio_mrp,amazon_mrp,amazon_fba_mrp,flipkart_mrp,limeroad_mrp,myntra_mrp,paytm_mrp,snapdeal_mrp
0,OS206_3141_S,OS206_3141,Moments,KURTA,0.3,538.0,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
1,OS206_3141_M,OS206_3141,Moments,KURTA,0.3,538.0,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
2,OS206_3141_L,OS206_3141,Moments,KURTA,0.3,538.0,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
3,OS206_3141_XL,OS206_3141,Moments,KURTA,0.3,538.0,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
4,OS206_3141_2XL,OS206_3141,Moments,KURTA,0.3,538.0,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0


### <br>Check the function ***wrangle_may2022() without null*** by printing head()

In [20]:
may2022_clean_without_null = wrangle_may2022(may2022,null = False)
may2022_clean_without_null.info()
may2022_clean_without_null.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sku             1221 non-null   object 
 1   style_id        1221 non-null   object 
 2   catalog         1221 non-null   object 
 3   category        1221 non-null   object 
 4   weight          1221 non-null   float64
 5   tp              1221 non-null   float64
 6   mrp_old         1221 non-null   float64
 7   final_mrp_old   1221 non-null   float64
 8   ajio_mrp        1221 non-null   float64
 9   amazon_mrp      1221 non-null   float64
 10  amazon_fba_mrp  1221 non-null   float64
 11  flipkart_mrp    1221 non-null   float64
 12  limeroad_mrp    1221 non-null   float64
 13  myntra_mrp      1221 non-null   float64
 14  paytm_mrp       1221 non-null   float64
 15  snapdeal_mrp    1221 non-null   float64
dtypes: float64(12), object(4)
memory usage: 152.8+ KB


,sku,style_id,catalog,category,weight,tp,mrp_old,final_mrp_old,ajio_mrp,amazon_mrp,amazon_fba_mrp,flipkart_mrp,limeroad_mrp,myntra_mrp,paytm_mrp,snapdeal_mrp
0,OS206_3141_S,OS206_3141,Moments,KURTA,0.3,538.0,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
1,OS206_3141_M,OS206_3141,Moments,KURTA,0.3,538.0,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
2,OS206_3141_L,OS206_3141,Moments,KURTA,0.3,538.0,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
3,OS206_3141_XL,OS206_3141,Moments,KURTA,0.3,538.0,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
4,OS206_3141_2XL,OS206_3141,Moments,KURTA,0.3,538.0,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0


### <br> Extract both the clean - 
1. dataframe with null to csv file named "***may2022_with_null_clean.csv***"
2. dataframe without null to csv file named "***may2022_without_null_clean.csv***"

In [21]:
may2022_clean_with_null.to_csv("may2022_with_null_clean.csv", index = False)
may2022_clean_without_null.to_csv("may2022_without_null_clean.csv", index = False)

### <br>Function for cleaning the dataset/dataframe **plmarch2021**

In [22]:
def wrangle_plmarch2021(dataframe,null = True):
    
    # copy dataframe to df
    df = dataframe.copy()
    
    # drop redundant col named "index"
    df.drop("index", axis = 1, inplace = True)
    
    # rename the col names
    df.columns = [element.replace(" ","_").lower() for element in df.columns]
    
    # values of "sku" and "stype_id" cols to upper case
    df["sku"] = df["sku"].str.upper()
    df["style_id"] = df["style_id"].str.upper()
    
    # dealing with "Nill" and "#VALUE!" entries in possible float-dtype columns
    
    # first: replace "Nill" and "#VALUE!" with integer 0 and change dtype to float
    df[['weight', 'tp_1', 'tp_2',
    'mrp_old','final_mrp_old',
    'ajio_mrp', 'amazon_mrp',
   'amazon_fba_mrp','flipkart_mrp',
    'limeroad_mrp','myntra_mrp',
    'paytm_mrp','snapdeal_mrp']] = df[['weight', 'tp_1', 'tp_2','mrp_old',
    'final_mrp_old', 'ajio_mrp', 'amazon_mrp',
    'amazon_fba_mrp','flipkart_mrp', 'limeroad_mrp',
    'myntra_mrp', 'paytm_mrp','snapdeal_mrp']].replace(["#VALUE!","Nill"],0).astype(float)
    
    # replace both 0 and "Nill" to "NaN" (null)
    df = df.replace([0,"Nill"],np.nan)
    
    # drop duplicates, if any
    df.drop_duplicates(inplace = True)
    
    # change the values in "category" col to uppercase
    df["category"] = df["category"].str.upper()
    
    # two return options: a dataframe with null and other dataframe without null
    if null:
        return df.reset_index(drop = True)
    else:
        return df.dropna().reset_index(drop = True)

 ### <br>Check the function ***wrangle_plmarch2021() with null*** by printing head()

In [23]:
plmarch2021_clean_with_null = wrangle_plmarch2021(plmarch2021,null = True)
plmarch2021_clean_with_null.info()
plmarch2021_clean_with_null.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330 entries, 0 to 1329
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sku             1330 non-null   object 
 1   style_id        1330 non-null   object 
 2   catalog         1257 non-null   object 
 3   category        1257 non-null   object 
 4   weight          1257 non-null   float64
 5   tp_1            1324 non-null   float64
 6   tp_2            1324 non-null   float64
 7   mrp_old         1293 non-null   float64
 8   final_mrp_old   1293 non-null   float64
 9   ajio_mrp        1293 non-null   float64
 10  amazon_mrp      1293 non-null   float64
 11  amazon_fba_mrp  1293 non-null   float64
 12  flipkart_mrp    1293 non-null   float64
 13  limeroad_mrp    1293 non-null   float64
 14  myntra_mrp      1299 non-null   float64
 15  paytm_mrp       1293 non-null   float64
 16  snapdeal_mrp    1293 non-null   float64
dtypes: float64(13), object(4)
memory 

,sku,style_id,catalog,category,weight,tp_1,tp_2,mrp_old,final_mrp_old,ajio_mrp,amazon_mrp,amazon_fba_mrp,flipkart_mrp,limeroad_mrp,myntra_mrp,paytm_mrp,snapdeal_mrp
0,OS206_3141_S,OS206_3141,Moments,KURTA,0.3,538.0,435.78,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
1,OS206_3141_M,OS206_3141,Moments,KURTA,0.3,538.0,435.78,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
2,OS206_3141_L,OS206_3141,Moments,KURTA,0.3,538.0,435.78,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
3,OS206_3141_XL,OS206_3141,Moments,KURTA,0.3,538.0,435.78,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
4,OS206_3141_2XL,OS206_3141,Moments,KURTA,0.3,538.0,435.78,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0


### <br>Check the function ***wrangle_plmarch2021() without null*** by printing head()

In [24]:
plmarch2021_clean_without_null = wrangle_plmarch2021(plmarch2021,null = False)
plmarch2021_clean_without_null.info()
plmarch2021_clean_without_null.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sku             1221 non-null   object 
 1   style_id        1221 non-null   object 
 2   catalog         1221 non-null   object 
 3   category        1221 non-null   object 
 4   weight          1221 non-null   float64
 5   tp_1            1221 non-null   float64
 6   tp_2            1221 non-null   float64
 7   mrp_old         1221 non-null   float64
 8   final_mrp_old   1221 non-null   float64
 9   ajio_mrp        1221 non-null   float64
 10  amazon_mrp      1221 non-null   float64
 11  amazon_fba_mrp  1221 non-null   float64
 12  flipkart_mrp    1221 non-null   float64
 13  limeroad_mrp    1221 non-null   float64
 14  myntra_mrp      1221 non-null   float64
 15  paytm_mrp       1221 non-null   float64
 16  snapdeal_mrp    1221 non-null   float64
dtypes: float64(13), object(4)
memory 

,sku,style_id,catalog,category,weight,tp_1,tp_2,mrp_old,final_mrp_old,ajio_mrp,amazon_mrp,amazon_fba_mrp,flipkart_mrp,limeroad_mrp,myntra_mrp,paytm_mrp,snapdeal_mrp
0,OS206_3141_S,OS206_3141,Moments,KURTA,0.3,538.0,435.78,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
1,OS206_3141_M,OS206_3141,Moments,KURTA,0.3,538.0,435.78,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
2,OS206_3141_L,OS206_3141,Moments,KURTA,0.3,538.0,435.78,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
3,OS206_3141_XL,OS206_3141,Moments,KURTA,0.3,538.0,435.78,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0
4,OS206_3141_2XL,OS206_3141,Moments,KURTA,0.3,538.0,435.78,2178.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0,2295.0


### <br> Extract both the clean - 
1. dataframe with null to csv file named "***plmarch2021_with_null_clean.csv***"
2. dataframe without null to csv file named "***plmarch2021_without_null_clean.csv***"

In [25]:
plmarch2021_clean_with_null.to_csv("plmarch2021_with_null_clean.csv", index = False)
plmarch2021_clean_without_null.to_csv("plmarch2021_without_null_clean.csv", index = False)

# Finished